## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-11-26-02-17-29 +0000,nypost,Illinois man finally exonerated after spendin...,https://nypost.com/2025/11/25/us-news/illinois...
1,2025-11-26-02-01-07 +0000,bbc,Parking inspector and wife arrested for €1m me...,https://www.bbc.com/news/articles/c997r8pyyz4o...
2,2025-11-26-02-00-00 +0000,wsj,Traders Are Flooding Markets With Risky Bets. ...,https://www.wsj.com/opinion/robinhood-ceo-vlad...
3,2025-11-26-01-58-00 +0000,wsj,New Zealand’s Central Bank Cuts Rates Further ...,https://www.wsj.com/economy/central-banking/rb...
4,2025-11-26-01-54-34 +0000,nyt,How Much Will Thanksgiving Dinner Cost? It Dep...,https://www.nytimes.com/2025/11/25/business/th...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2331/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
59,trump,44
68,ukraine,22
69,peace,18
25,new,17
125,year,14


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
7,2025-11-26-01-40-51 +0000,nypost,Top Trump aide Steve Witkoff coached Russians ...,https://nypost.com/2025/11/25/world-news/top-t...,123
261,2025-11-25-12-56-44 +0000,nypost,"Trump Army secretary meets Russians after US, ...",https://nypost.com/2025/11/25/us-news/trump-ar...,122
78,2025-11-25-22-46-23 +0000,bbc,Ukraine says 'understanding' reached with US o...,https://www.bbc.com/news/articles/cy95jvw57v2o...,120
155,2025-11-25-20-03-41 +0000,nypost,Trump dispatches top envoys to Russia and Ukra...,https://nypost.com/2025/11/25/us-news/trump-sa...,120
181,2025-11-25-18-50-55 +0000,nypost,How Army Secretary Dan Driscoll rose from Trum...,https://nypost.com/2025/11/25/us-news/how-army...,120


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
7,123,2025-11-26-01-40-51 +0000,nypost,Top Trump aide Steve Witkoff coached Russians ...,https://nypost.com/2025/11/25/world-news/top-t...
338,61,2025-11-25-03-24-31 +0000,nypost,Initial Ukraine-Russia peace plan was hashed o...,https://nypost.com/2025/11/24/us-news/initial-...
9,48,2025-11-26-01-25-44 +0000,bbc,New Zealand woman gets life sentence for 'suit...,https://www.bbc.com/news/articles/cp84xed0g18o...
127,47,2025-11-25-21-14-00 +0000,wsj,At the White House’s annual—and typically stai...,https://www.wsj.com/politics/policy/trump-pard...
24,45,2025-11-26-00-42-29 +0000,nypost,Fashionable fliers take off with Sean Duffy’s ...,https://nypost.com/2025/11/25/us-news/fliers-p...
273,41,2025-11-25-12-00-00 +0000,missionlocal,Trump budget will drain $315M a year from San ...,https://missionlocal.org/2025/11/sf-trump-budg...
35,40,2025-11-26-00-09-20 +0000,nypost,Football coach Travis Turner is just latest st...,https://nypost.com/2025/11/25/us-news/coach-tr...
114,32,2025-11-25-21-36-00 +0000,wsj,Autodesk CEO Sees Long-Term Growth Due to Stea...,https://www.wsj.com/business/earnings/autodesk...
28,29,2025-11-26-00-32-00 +0000,wsj,A group of Democratic lawmakers said the FBI h...,https://www.wsj.com/politics/policy/democratic...
55,28,2025-11-25-23-22-59 +0000,nypost,Brooklyn’s dine-and-dash diva may face deporta...,https://nypost.com/2025/11/25/us-news/brooklyn...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
